In [3]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys
import re
from glob import glob
import csv

import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError

from humanfriendly import prompts

from progressbar import AnimatedMarker, Bar, BouncingBar, Counter, ETA, \
    AdaptiveETA, FileTransferSpeed, FormatLabel, Percentage, \
    ProgressBar, ReverseBar, RotatingMarker, \
    SimpleProgress, Timer, UnknownLength

# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [2]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [4]:
def fileSearch(path = None, search = None):
        '''search for a file name string in a given path'''
        if not search:
            search = ''
        logger = logging.getLogger(__name__)
        logger.debug('searching path: {0} for \"{1}\"'.format(path, search))
        try:
            searchPath = os.path.expanduser(path)
        except AttributeError as e:
            logger.error('Error: no path specified')
            searchPath = ''
        try:
            allFiles = os.listdir(searchPath)
        except OSError as e:
            logger.error(e)
            return(None)
        
        # list comprehension search with regex
        #http://www.cademuir.eu/blog/2011/10/20/python-searching-for-a-string-within-a-list-list-comprehension/
        regex = re.compile('.*('+search+').*')
        return([m.group(0) for l in allFiles for m in [regex.search(l)] if m])

In [5]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.debug('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [6]:
def getTeamDrive(myDrive):
    '''
    menu driven interaction for choosing a writeable team drive
    '''
    #available and writeable team drives
    logger = logging.getLogger(__name__)
    try:
        teamdrives_all = myDrive.listTeamDrives()
    except Exception as e:
        logger.error(e)
    
    if len(teamdrives_all) < 1:
        logger.warn('no writeable team drives found for this user')
        return(None)
    
    teamdrives_writeable = {}
    for drive in teamdrives_all:
        if drive['capabilities']['canAddChildren']:
            teamdrives_writeable[drive['name']] = drive
    
    teamdrive = None
    
    try:
        teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None)
        teamdrive=teamdrives_writeable[teamdrive_name]
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
    
    return(teamdrive)

In [7]:
def getPortfolioFolder(myDrive):
    '''
    menu driven interaction to locate and choose portfolio folder on team drive
    
    returns:
        folder (dictionary) - google drive object properties dictionary
    '''
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive: {}; {}'.format(teamdriveName, teamdriveID))
    
    foldersearch = ''
    attempt = 5
    while (len(foldersearch) <= 0) and (attempt > 0):
        attempt -= 1
        foldersearch = prompts.prompt_for_input(question='Please enter a portion of the portfolio folder name (case insensitive): ',
                                                default=None, strip=True)
        if len(foldersearch) <= 0:
            print 'Nothing entered. {} attempts remain.'.format(attempt)
            
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        if len(searchresult['files']) <= 0:
            print 'No folders contained "{}"'.format(foldersearch)
            print 'Please try entering only part of the folder name'
            foldersearch = ''
            
    
    
    if foldersearch <= 0:
        logger.warn('user entered 0 length string for folder search')
        return(None)
    
    
    foldernames = []
    for each in searchresult['files']:
        foldernames.append(each['name'])
    
    try:
        foldername = prompts.prompt_for_choice(choices=foldernames)
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
        return(None)
    
    return(searchresult['files'][foldernames.index(foldername)])
    

In [8]:
def getPathfromList(list_path=['~/'], message='Choose from the paths below', default=None):
    '''
    menu driven interaction to select path 
    accepts:
        list_path (list) - list of paths to display
        messager (string) - message to display
        default (string) - default choice (must be one of the items in the list otherwise defaults to None)
    
    returns:
        searchPath (string) - valid and existing path
    '''
    logger = logging.getLogger(__name__)
    logger.debug('path list: {}'.format(list_path))
    searchPath = None
    list_path = sorted(list_path)
    list_path.append('OTHER')
    
    if default not in list_path:
        logging.warn('default ({}) not in list; setting default = None'.format(default))
        default = None
    
    if message:
        print message
    searchPath = prompts.prompt_for_choice(choices=list_path, default=default)
    logger.debug('searchPath = {}'.format(searchPath))
    
    if not searchPath is 'OTHER':
        searchPath = os.path.expanduser(searchPath)
    else:
        attempt = 5
        searchPath = None
        while not searchPath and attempt > 0:
            logger.debug('attempts remaining: {}'.format(attempt))
            attempt -= 1
            searchPath = prompts.prompt_for_input('Please type the complete path to use: ')
            logger.debug('searchPath = {}'.format(searchPath))         
    
    if len(searchPath) < 1:
        searchPath = None
    return (searchPath) 

In [9]:
def getFiles(path='~/', pattern='.*', ignorecase=True):
    '''
    search path for files matching pattern (regular expression)
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case'''
    logger = logging.getLogger(__name__)
    
    path = os.path.expanduser(path+'/*')
    logger.debug('Path: {}, glob pattern: {}, ignorecase: {}'.format(path, pattern, ignorecase))
    
    flags = re.IGNORECASE
    files = []
        
    if ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f, flags=flags)]:
            files.append(eachfile)
    
    if not ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f)]:
            files.append(eachfile)        
    
    return(files)

In [10]:
def chooseFile(path='~/', pattern='.*', ignorecase=True, message='Please choose a file from the list'):
    '''
    menu interaction for choose a file in the specified path from the file glob created using a regex pattern
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case
        message(string) - message to show
    returns:
        selection (string) - selected file'''
    
    logger = logging.getLogger(__name__)
    filelist = getFiles(path=path, pattern=pattern, ignorecase=ignorecase)
    logger.debug('filelist: {}'.format(filelist))
    print message
    selection = prompts.prompt_for_choice(choices=filelist)
    return(selection)

In [11]:
def fileToList(inputfile):
    logger = logging.getLogger(__name__)
    logger.debug('inputfile = {}'.format(inputfile))
    outputlist = []
    
    try:
        with open(inputfile, 'r') as fhandle:
            logger.debug('reading file: {}'.format(inputfile))
            for line in fhandle:
                outputlist.append(line.strip('\n'))
    except IOError as e:
        logger.debug(e)
    return(outputlist)

In [12]:
def checkFolder(folderID, myDrive):
    '''
    Checks properties of a folder id

    returns:
        tuple(isFolder (bool), writeable(bool), properties(dict)) - false if no match
    
    '''
    logger = logging.getLogger(__name__)
    logger.debug('checking folder id: {}'.format(folderID))
    props = None
    isFolder = False
    writeable = False
    try:
        logger.debug('checking properties for ID: {}'.format(folderID))
        props = myDrive.getprops(folderID, 'capabilities, mimeType, name')
    except GDriveError as e:
        logger.error('failed to get properties for {}; {}'.format(folderID, e))
        logger.error('please try again. if this error persists, please try reconfiguring the folder')
        logger.error('cannot continue; exiting')
    
    try:
        if props['mimeType'] == myDrive.mimeTypes['folder']:
            isFolder = True
            logger.debug('this ID is a folder')
        if props['capabilities']['canAddChildren']:
            writeable = True
            logger.debug('this ID is a writable folder')
        else:
            logger.debug('this ID is not a writable folder')
    except (KeyError, TypeError) as e:
        pass
    
    if not props:
        logger.warn('this ID is not a folder')
                
    return(isFolder, writeable, props)

In [13]:
def mapHeaders(file_csv, expected_headers=[]):
    '''map an expected list of header values to their position in a csv
    accepts:
        file_csv (csv.reader object) 
        expected_headers (list) - list of headers to search for, ignoring all others
    '''
    logger = logging.getLogger(__name__)
    logger.debug('mapping headers')
    
    missing_headers = []
    headerMap = {} 
    
    try:
        csvHeader = file_csv[0]
    except IndexError as e:
        logging.error('csv empty: {}'.format(e))
        return(None)
    
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
    
    headerMap['missingheaders'] = missing_headers
    
    headerMap['headers'] = {}

    if len(missing_headers) > 0:
        logging.warn('missing headers: {}'.format(missing_headers))
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            headerMap['headers'][value] = index
    logger.debug('successfully maped headers')
    return(headerMap)

In [14]:
def doExit(exit_level=0, testing=False):
    logger.info('exiting before completion with exit code {}'.format(exit_level))
    if not testing:
        exit(0)    

In [15]:
def createFolders(myDrive, teamdrive, parentFolder, folderList, gradefolder_list, progressbar=True):
    '''
    create folders folders in a valid parent folder from a list
    '''
    createdFolders = {}
    logger = logging.getLogger(__name__)
    logger.debug('creating folders')
    logger.debug('checking parent folder: {}'.format(parentFolder))
    
    # Define progress bar widgets for consistent look and feel
    widgets = ['Checking: ', Percentage(), ' ',
               Bar(marker='=',left='[',right=']'), 
               ' Processed ', Counter(),
               ' ', ETA()]
    
    try:
        folder_check = checkFolder(parentFolder, myDrive)
    except GDriveError as e:
        logger.error('error checking parent folder: {}'.format(e))
    except Exception as e:
        logger.error('critical error: {}'.format(e))
    if not all(folder_check):
        try:
            mimeType = folder_check[2]['mimeType']
        except (TypeError, KeyError) as e:
            mimeType = 'Unknown'
            return(None)
        logger.warn('parent folder cannot be used\n is folder: {}\n is writeable: {}\n mimeType: {}'.format(folder_check[0], folder_check[1], mimeType))
        return(None)

    # init the ProgressBar
    if progressbar:
        pbar = ProgressBar(widgets=widgets, maxval=len(folderList))
        pbar.start()
        pbar_index = 0
        print 'Processing {} folders'.format(len(folderList))

    for folder in folderList:
        # update the progress bar
        if progressbar:
            pbar.update(pbar_index)
            pbar_index += 1
        
        # expect folder should be created
        createFolder = True
        
        # check if folder already exists
        logger.debug('searching for exisitng folder with name: {}'.format(folder))
        
        try:
            result = myDrive.search(parents=parentFolder, name=folder, trashed=False, mimeType='folder', 
                                    orderBy='createdTime', teamdrive=teamdrive)
        except GDriveError as e:
            logger.error('error searching for folder: {}'.format(e))
            createdFolders[folder] = None
        except Exception as e:
            logger.error('critical error: {}'.format(e))
            logger.error('skipping folder: {}'.format(folder))
            continue
        
        if result.has_key('files'): 
            if len(result['files']) > 0:
                logger.debug('folder already exists, do not create')
                createFolder = False
                createdFolders[folder] = result['files'][0]
            if len(result['files']) > 1:
                logger.warn('{} folders with the same name ({}) found. Consider removing the newest one'.format(len(result['files']), folder))
        else:
            createFolder = True
            
        if createFolder:
            try:
                result = myDrive.add(name=folder, parents=parentFolder, mimeType='folder')
            except GDriveError as e:
                logger.error('error creating folder: {}'.format(e))
                createdFolders[folder] = None
            except Exception as e:
                logger.error('critical error: {}'.format(e))
                logger.error('skipping folder: {}'.format(folder))
                continue
            
            createdFolders[folder] = result
            
    if progressbar:
        pbar.finish()
        print 'Completed\n'
    
    return(createdFolders)

In [ ]:
myList = ['Class Of-2029', 'Class Of-2030', 'Class Of-2019']
foof = createFolders(myDrive=myDrive, teamdrive=teamdriveID, folderList=myList, parentFolder='1-D7UNBes_skfkQ6oBettiBICiBcZmbvn')

In [50]:
def createPortfolioFolders(myDrive, parentFolder, teamdriveID, studentexport_csv, headerMap):
    '''
    create folders on google drive if they do not exist
    
    accepts:
        myDrive(googleDrive object)
        folderid (string) - folder id string
        folders (list) - list of folder names
        studentexport_csv - list of lists 
        headerMap - map of fields
    
    returns:
        folderinfo (dict) - {folder name: folder url, or None for failures}
    '''
    # progress bar widgets
    widgets = ['Checking: ', Percentage(), ' ',
               Bar(marker='=',left='[',right=']'), 
               ' Processed ', Counter(),
               ' ', ETA()]
    
    logger = logging.getLogger(__name__)
    logger.debug('creating folders at gdrive path: {}'.format(folderID))
    
    studentFolders = {}
    classOfFolders = {}
    classOf_string = 'Class Of-'
    
    # check that google drive parent folder exists and is a folder
    try:
        folder_check = checkFolder(parentFolder, myDrive)
    except GDriveError as e:
        logger.error('error checking parent folder: {}'.format(e))
    except Exception as e:
        logger.error('critical error: {}'.format(e))
    if not all(folder_check):
        try:
            mimeType = folder_check[2]['mimeType']
        except (TypeError, KeyError) as e:
            mimeType = 'Unknown'
            return(None)
        logger.warn('parent folder cannot be used\n is folder: {}\n is writeable: {}\n mimeType: {}'.format(folder_check[0], folder_check[1], mimeType))
        return(None)

    
    # gather all unique and classOf rows from csv and build dictionaries
    logger.debug('gathering "Class Of" folders from student export file')
  
    # build dictionaries 
    for student in studentexport_csv[1:]:
        classOf = student[headerMap['headers']['ClassOf']]
        studentNumber = student[headerMap['headers']['Student_Number']]
        lastFirst = student[headerMap['headers']['LastFirst']]
        name = lastFirst + ' - ' + studentNumber
        
        classOfFolders[classOf] = {}
        classOfFolders[classOf]['name'] = classOf_string + classOf
        
        studentFolders[studentNumber] = {}
        studentFolders[studentNumber]['name'] = name
        studentFolders[studentNumber]['classOf'] = classOf
    
    if len(classOfFolders) <1:
        print 'No valid rows were found in CSV; cannot continue'
        logger.error('no valid rows found in CSV; cannot continue')
        return(None)
    logger.debug('{} "Class Of" folders found in student export file'.format(len(classOfFolders)))
    logger.debug('{} student records found in student export file'.format(len(studentFolders)))
    
    
    # check for existence of Class Of folders and create if they are missing
    logger.debug('checking for existing "Class Of" folders in google drive')
    
    print 'Checking for or creating {} "Class Of-" folders in portfolio folder'.format(len(classOfFolders))
    classOfFolders = createFolders(myDrive=myDrive, parentFolder=parentFolder, 
                                   teamdrive=teamdriveID,
                                   folderList=[value.get('name') for key, value in classOfFolders.items()])
    
        
        
        
    # create student folders within the class of folders
    
    logger.debug('{} students found in student export file'.format(len(studentexport_csv)-1))
    
    # check for existence of Class Of folders and create if they are missing
    logger.debug('checking for existing student folders in google drive')

    print 'Checking for or creating {} student folders in portfolio folder'.format(len(studentexport_csv)-1)
    

    classOfFolders = createFolders(myDrive=myDrive, parentFolder=parentFolder, 
                                   teamdrive=teamdriveID,
                                   folderList=[value.get('name') for key, value in classOfFolders.items()])

    # init progress bar for class of folders 
    print 'Checking for or creating {} student folders in "Class Of-" folders'.format(len(studentexport_csv)-1)
    pbar = ProgressBar(widgets=widgets, maxval=len(studentexport_csv)-1)
    pbar.start()
    pbar_index = 0
    
    for student in studentexport_csv[1:]:
        # update the progress bar
        pbar.update(pbar_index)
        pbar_index += 1
      
        # start off assuming the folder needs to be created
        createFolder = True
        classOf = student[headerMap['headers']['ClassOf']]
        student_number = student[headerMap['headers']['Student_Number']]
        student_LastFirst = student[headerMap['headers']['LastFirst']]
        folder_name =  student_LastFirst + ' - ' + student_number
        studentFolders[student_number] = {}

        # use the classOf folder ID for the parent
        if classOfFolders['Class Of-'+str(classOf)]:
            classOf_id = classOfFolders['Class Of-'+str(classOf)]['id']
        else:
            logger.error('missing data for parent folder: Class Of-{}; skipping student'.format(classOf))
            studentFolders[student_number] = None
            continue
        
        
        # check if folder already exists
        logger.debug('searching for exisitng folder with name: {}'.format(folder_name))
        
        try:
            result = myDrive.search(parents=classOf_id, name=folder_name, trashed=False, mimeType='folder', 
                                    orderBy='createdTime', teamdrive=teamdriveID)
        except GDriveError as e:
            logger.error('error searching for folder: {}'.format(e))
            createdFolders[folder] = None
        except Exception as e:
            logger.error('critical error: {}'.format(e))
            logger.error('skipping folder: {}'.format(folder_name))
            continue
        
        if result.has_key('files'): 
            if len(result['files']) > 0:
                logger.debug('folder already exists, do not create')
                createFolder = False
                # use the first in the array
                studentFolders[student_number] = result['files'][0]
                                                        
            if len(result['files']) > 1:
                logger.warn('{} folders with the same name ({}) found. Consider removing the newest one'.format(len(result['files']), folder))

                
            print '*'*30
            print 'search result:\n',result
                
        if createFolder:
            logger.debug('creating folder: {}'.format(folder_name))
            try:
                result = myDrive.add(name=folder_name, parents=classOf_id, mimeType='folder',
                                    fields='name, webViewLink, id, mimeType')
            except GDriveError as e:
                logger.error('error creating folder {}: {}'.format(folder_name, e))
                logger.error('skipping folder')
                studentFolders[student_number] = None
                continue
            except Exception as e:
                logger.error('critical error: {}'.format(e))
                studentFolders[student_number] = None
                logger.error('skipping folder')
            

            
            # check the result is OK and record it
            if any(result):
                studentFolders[student_number] = result
            else:
                studentFolders[student_number] = None
                continue
            
        # append the LastFirst field to the dictionary
        studentFolders[student_number]['LastFirst'] = student_LastFirst
        studentFolders[student_number]['classOf'] = classOf
        
        # create the grade-level folders if needed
        

    # finish updating the pbar for Class Of- Folders
    pbar.finish()
    print 'Completed\n'
        
    return(studentFolders)

In [51]:
myResult = createPortfolioFolders(myDrive, parentFolder='1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', teamdriveID=teamdriveID, studentexport_csv=studentexport_csv, headerMap=headerMap)
print myResult

2018-09-26 20:27:20,942: [DEBUG: __main__.createPortfolioFolders] creating folders at gdrive path: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-26 20:27:20,943: [DEBUG: __main__.checkFolder] checking folder id: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-26 20:27:20,943: [DEBUG: __main__.checkFolder] checking properties for ID: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-26 20:27:20,944: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:20,945: [DEBUG: gdrive.gdrive.getprops] files().get(fileId=1-D7UNBes_skfkQ6oBettiBICiBcZmbvn, fields=capabilities,mimeType,name)
2018-09-26 20:27:20,951: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files/1-D7UNBes_skfkQ6oBettiBICiBcZmbvn?fields=capabilities%2CmimeType%2Cname&alt=json&supportsTeamDrives=true
2018-09-26 20:27:21,176: [DEBUG: __main__.checkFolder] this ID is a folder
2018-09-26 20:27:21,177: [DEBUG: __main__.checkFolder] this ID is a writable folder
2018-09-26 20:27:21,178: [DEBUG: 

Processing 4 folders
2018-09-26 20:27:21,396: [DEBUG: __main__.createFolders] searching for exisitng folder with name: Class Of-2019
2018-09-26 20:27:21,397: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:21,398: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2019" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:21,403: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2019%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-26 20:27:21,719: [DEBUG: __main__.createFolders] folder already exists, do not create


2018-09-26 20:27:21,720: [DEBUG: __main__.createFolders] searching for exisitng folder with name: Class Of-2027
2018-09-26 20:27:21,721: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:21,722: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2027" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:21,727: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2027%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-26 20:27:22,259: [DEBUG: __main__.createFolders] folder already exists, do not create


2018-09-26 20:27:22,261: [DEBUG: __main__.createFolders] searching for exisitng folder with name: Class Of-2028
2018-09-26 20:27:22,262: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:22,263: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2028" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:22,268: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2028%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-26 20:27:22,570: [DEBUG: __main__.createFolders] folder already exists, do not create


2018-09-26 20:27:22,571: [DEBUG: __main__.createFolders] searching for exisitng folder with name: Class Of-2029
2018-09-26 20:27:22,572: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:22,573: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2029" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1-D7UNBes_skfkQ6oBettiBICiBcZmbvn" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:22,578: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2029%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221-D7UNBes_skfkQ6oBettiBICiBcZmbvn%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-26 20:27:23,050: [DEBUG: __main__.createFolders] folder already exists, do not create


...

2018-09-26 20:27:39,741: [DEBUG: __main__.createPortfolioFolders] searching for exisitng folder with name: Borg, William - 503955
2018-09-26 20:27:39,742: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:39,743: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Borg, William - 503955" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:39,748: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Borg%2C+William+-+503955%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
******************************
search result:
{u'files': []}
2018-09-26 20:27:40,258: [

2018-09-26 20:27:41,041: [DEBUG: __main__.createPortfolioFolders] searching for exisitng folder with name: Sue, Sally - 000000
2018-09-26 20:27:41,042: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 20:27:41,043: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Sue, Sally - 000000" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1WIGcKU0w9JDEYfADHSijduGuL2Yf06Nc" in parents, orderBy=createdTime, fields=files))
2018-09-26 20:27:41,048: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Sue%2C+Sally+-+000000%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221WIGcKU0w9JDEYfADHSijduGuL2Yf06Nc%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-26 20:27:41,583: [DEBUG: __main__.createPortfolioFolders] folder already exists, do not 

******************************
search result:
{u'files': [{u'mimeType': u'application/vnd.google-apps.folder', u'hasThumbnail': False, u'modifiedByMeTime': u'2018-09-22T20:20:51.098Z', u'thumbnailVersion': u'0', u'explicitlyTrashed': False, u'isAppAuthorized': True, u'viewedByMeTime': u'2018-09-23T06:33:55.479Z', u'id': u'1n46pSRak26sPqoXZLXlGYKMY12uEv4g7', u'lastModifyingUser': {u'me': True, u'kind': u'drive#user', u'displayName': u'Aaron Ciuffo', u'permissionId': u'14867985035974787748', u'emailAddress': u'aciuffo@ash.nl', u'photoLink': u'https://lh3.googleusercontent.com/-h73ELuq3EPc/AAAAAAAAAAI/AAAAAAAAABg/USQhYvMjp5k/s64/photo.jpg'}, u'capabilities': {u'canUntrash': True, u'canMoveItemWithinTeamDrive': True, u'canDeleteChildren': True, u'canMoveChildrenWithinTeamDrive': True, u'canListChildren': True, u'canRename': True, u'canAddChildren': True, u'canShare': True, u'canTrashChildren': True, u'canReadRevisions': True, u'canCopy': True, u'canMoveItemIntoTeamDrive': True, u'canMoveIt

******************************
search result:
{u'files': [{u'mimeType': u'application/vnd.google-apps.folder', u'hasThumbnail': False, u'modifiedByMeTime': u'2018-09-22T20:20:52.028Z', u'thumbnailVersion': u'0', u'explicitlyTrashed': False, u'isAppAuthorized': True, u'id': u'1DOlchPt0BjCGzTKIramTxImV0jXeICvj', u'lastModifyingUser': {u'me': True, u'kind': u'drive#user', u'displayName': u'Aaron Ciuffo', u'permissionId': u'14867985035974787748', u'emailAddress': u'aciuffo@ash.nl', u'photoLink': u'https://lh3.googleusercontent.com/-h73ELuq3EPc/AAAAAAAAAAI/AAAAAAAAABg/USQhYvMjp5k/s64/photo.jpg'}, u'capabilities': {u'canUntrash': True, u'canMoveItemWithinTeamDrive': True, u'canDeleteChildren': True, u'canMoveChildrenWithinTeamDrive': True, u'canListChildren': True, u'canRename': True, u'canAddChildren': True, u'canShare': True, u'canTrashChildren': True, u'canReadRevisions': True, u'canCopy': True, u'canMoveItemIntoTeamDrive': True, u'canMoveItemOutOfTeamDrive': True, u'canComment': True, u'

******************************
search result:
{u'files': [{u'mimeType': u'application/vnd.google-apps.folder', u'hasThumbnail': False, u'modifiedByMeTime': u'2018-09-26T17:45:18.498Z', u'thumbnailVersion': u'0', u'explicitlyTrashed': False, u'isAppAuthorized': True, u'viewedByMeTime': u'2018-09-26T17:45:59.024Z', u'id': u'158lTISLZRERR7ubwALtTx_0T3zAh6yor', u'lastModifyingUser': {u'me': True, u'kind': u'drive#user', u'displayName': u'Aaron Ciuffo', u'permissionId': u'14867985035974787748', u'emailAddress': u'aciuffo@ash.nl', u'photoLink': u'https://lh3.googleusercontent.com/-h73ELuq3EPc/AAAAAAAAAAI/AAAAAAAAABg/USQhYvMjp5k/s64/photo.jpg'}, u'capabilities': {u'canUntrash': True, u'canMoveItemWithinTeamDrive': True, u'canDeleteChildren': True, u'canMoveChildrenWithinTeamDrive': True, u'canListChildren': True, u'canRename': True, u'canAddChildren': True, u'canShare': True, u'canTrashChildren': True, u'canReadRevisions': True, u'canCopy': True, u'canMoveItemIntoTeamDrive': True, u'canMoveIt

In [26]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))

studentexport_list = ['~/Downloads', '~/Desktop']
studentexport = None
studentexport_path = None


downloadlink = 'https://foo.foof.foo'


updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

# start with this in case it is never properly set
teamdriveName = '**UNKNOWN**'

# for use this for doExit to prevent killing the Jupyter kernel when testing
testing = True

# get the log file names
log_files = []
for handlers in logger.root.handlers:
    if 'FileHandler' in str(handlers.format.im_class):
        log_files.append(handlers.baseFilename)

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = './gradefolders.txt'


    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logger.debug('current configuration:')
    logger.debug('\n{}'.format(config_dict))

# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account with access to the portfolio folder team drive and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
if not prompts.prompt_for_confirmation(question='Would you like to proceed?', default='y'):
    print 'Exiting'
    doExit(testing=testing)
    
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if not teamdriveID:
    logger.warn('Team Drive ID missing')
    try:
        teamdrive = getTeamDrive(myDrive)
    except GDriveError as e:
        logging.error('Error accessing team drives: e')
        doExit(testing=testing)
        
    if not teamdrive:
        logger.error('no team drives found')
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print('No Team Drives located; cannot proceed.')
        print('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        logger.error('exiting')
        doExit(testing=testing)
        
    try:
        teamdriveID = teamdrive['id']
        teamdriveName = teamdrive['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid team drive information found: {}'.format(e))
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print 'No Team Drive was found. Do you have write access to a Team Drive with this account?'
        print 'You are authenticated as: {}'.format(myDrive.userinfo['emailAddress'])
        logger.error('exiting')
        doExit(testing=testing)
        
    print 'Using Team Drive: {}'.format(teamdriveName)    
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdriveName', teamdriveName)
    updateConfig = True
    
if not folderID:
    logger.warn('Folder ID missing')
    try:
        folder = getPortfolioFolder(myDrive)
    except GDriveError as e:
        logger.error('Could not search for folder: {}'.format(e))
        print ('There was a problem searching Team Drive for a folder')
        print ('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        print ('If this is not correct do...')
    try:
        folderID = folder['id']
        folderName = folder['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid folder information found {}'.format(e))
        logger.error('exiting')
        doExit(testing=testing)
    
    print 'Using Folder: {} on Team Drive: {}'.format(folderName, teamdriveName)
    myConfig.set('Main', 'foldername', folderName)
    myConfig.set('Main', 'folderid', folderID)
    updateConfig=True

if not os.path.exists(gradefolders):
    logger.error('failed to locate {}'.format(gradefolders))
    logger.error('exiting')
    print 'gradefolders.txt file is missing. Please reinstall the application.'
    print 'download link: {}'.format(downloadlink)
    doExit(testing=testing)

# get the gradefolder file
gradefolder_list = fileToList(gradefolders)

# check validity of gradefolder
if len(gradefolder_list) < 1:
    print 'grade folder file is corrupt. Please reinstall the application'
    doExit(testing=testing)


if updateConfig:
    configuration.create_config(cfgfile, myConfig)


# # get and validate path for student_export folder
attempt = 5
while not studentexport and (attempt > 0):
    logger.debug('remaining attempts to locate student export: {}'.format(attempt))
    attempt -= 1
    if not studentexport:
        # invoke menu for prompting user to choose a path to the studentexport file
        studentexport_path = getPathfromList(studentexport_list, 
                                             message='Please choose which folder contains the Student Export File.',
                                             default=studentexport_list[0])
    
    # move this pattern into the configuration file? 
    studentexport = chooseFile(path=studentexport_path, pattern='.*student.*export.*')
    logger.debug('student export file chosen: {}'.format(studentexport))
    if not os.access(studentexport, os.R_OK):
        print 'Could not read file: {}. Please choose another file'.format(studentexport)
        logger.error('file is unreadable: {}'.format(studentexport))
        studentexport = None

#TO DO - this looks odd. it appears that studentexport is pulled twice but unused.
# try commenting this entire block out. It does not look right.
if not studentexport:
    logger.error('no student export file; exiting')
    print 'Cannot proceed without a student export file. Exiting'
    doExit(testing=testing)
else:
    myConfig.set('Main', 'studentexport', studentexport)
    studentexport_list = fileToList(studentexport)
        
logger.debug('Student export file: {}'.format(studentexport))

# read the studentexport text file into a csv object
studentexport_csv = []
try: 
    with open(studentexport, 'rU') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            studentexport_csv.append(row)
except (OSError, IOError) as e:
    logging.critical('error reading file: {}\n{}'.format(studentexport,e))
    print 'Could not read file: {}; exiting.'.format(studentexport)
    doExit(testing=testing)

# parse and check validity of student export file
expected_headers = ['ClassOf', 'Student_Number', 'LastFirst']
headerMap = mapHeaders(file_csv=studentexport_csv, expected_headers=expected_headers)

if not headerMap or (len(headerMap['headers']) != len(expected_headers)):
    logging.error('cannot continue without valid header map; exiting')
    print('file: {} appears to not contain the expected header row (in any order): {}'.format(studentexport, expected_headers))
    if headerMap['missingheaders']:
        print('your file appears to be missing the field(s): {}'.format(headerMap['missingheaders']))
    print('please try to download a new student.export file before trying again; exiting')
    doExit(testing=testing)

logger.debug('asking to proceed with current configuration')
print '\nCurent Configuration:'
for section in myConfig.sections():
    print '[{}]'.format(section)
    for option in myConfig.options(section):
        print '{} = {}'.format(option, myConfig.get(section, option))
if not prompts.prompt_for_confirmation('Continue with the configuration above?', default='Y'):
    logger.debug('user chose to exit')
    doExit(testing=testing)
    
# # start processing the student export file
# studentFolders = createFolders(myDrive=myDrive, parentFolder=folderID, teamdriveID=teamdriveID, 
#                                 studentexport_csv=studentexport_csv, header_map=headerMap )

# if not studentFolders:
#     logger.error('Student folders were not created. Please try again. If this message persists, please check the logs')
#     doExit(testing=testing)

# # foo['100000'] = None
# # failures = 0
# # for each in foo:
# #     if not foo[each]:
# #         print 'student not processed due to errors: {}'.format(each)
# #         failures += 1
# # if failures > 0:
# #     print 'Please re-run the program with the same data. If this message appears again, please check the logs [[path to logs]]'


# failures = 0
# for eachStudent in StudentFolders:
#     if not studentFolders[eachStudent]:
#         logger.warn('student not processed due to previous errors: {}'.format(eachStudent))
#         print('Please run this batch again. If this message persists, please check the logs')

# # urlFormatter = '<a href={}>Student Portfolio for {}</a>'
# # studentData_header = ['LastFirst', 'Student_Number', 'StudentURL']
# # studentData_list = []
# # studentData_list.append(studentData_header)

# # for each in foo:
# #     if foo[each]:
# #         # LastFirst, Student Number, URL
# #         studentData_list.append([foo[each]['LastFirst'], each, 
# #                                  urlFormatter.format(foo[each]['webViewLink'], foo[each]['LastFirst'])])
        


# # with open('./foo.tsv', mode='w') as tsv_file:
# #     csv_writer = csv.writer(tsv_file, delimiter='\t')
# #     csv_writer.writerows(studentData_list)

2018-09-26 19:44:41,490: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-09-26 19:44:41,491: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-09-26 19:44:41,494: [DEBUG: __main__.<module>] current configuration:
2018-09-26 19:44:41,495: [DEBUG: __main__.<module>] 
{'Main': {'folderid': '1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', 'credentials': '~/.config/portfolio_creator/credentials/', 'foldername': 'Portfolios', 'teamdrivename': 'IT Blabla', 'teamdriveid': '0ACLfU8KeD_BHUk9PVA'}}
2018-09-26 19:44:41,495: [INFO: root.<module>] checking google credentials




You *may* be directed to a google web page that asks you to authorize this applicaiton.
Please choose an ASH account with access to the portfolio folder team drive and authorize this applicaiton.
When you are done, please close the newly created tab in your web browser and return to this window.
2018-09-26 19:44:41,497: [DEBUG: humanfr

2018-09-26 19:44:42,718: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-26 19:44:42,721: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/teamdrives?fields=teamDrives&alt=json
2018-09-26 19:44:43,044: [DEBUG: __main__.fileToList] inputfile = ./gradefolders.txt
2018-09-26 19:44:43,045: [DEBUG: __main__.fileToList] reading file: ./gradefolders.txt
2018-09-26 19:44:43,053: [DEBUG: __main__.<module>] remaining attempts to locate student export: 5
2018-09-26 19:44:43,054: [DEBUG: __main__.getPathfromList] path list: ['~/Downloads', '~/Desktop']
Please choose which folder contains the Student Export File.
2018-09-26 19:44:43,055: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are '~/Desktop', '~/Downloads' and 'OTHER') ..

  1. ~/Desktop
  2. ~/Downloads (default choice)
  3. OTHER
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-26 19:44:44,912: [DEBUG:


  1. /Users/aaronciuffo/Downloads/student.export (1).text
  2. /Users/aaronciuffo/Downloads/student.export (2).text
  3. /Users/aaronciuffo/Downloads/student.export (3).text
  4. /Users/aaronciuffo/Downloads/student.export (4).text
  5. /Users/aaronciuffo/Downloads/student.export.text
  6. /Users/aaronciuffo/Downloads/student_export.text
  7. /Users/aaronciuffo/Downloads/Student_Export.txt
 
 Enter your choice as a number or unique substring (Control-C aborts): 4
2018-09-26 19:44:46,708: [DEBUG: humanfriendly.prompts.prompt_for_choice] Option ('/Users/aaronciuffo/Downloads/student.export (4).text') selected by numeric reply (4).
2018-09-26 19:44:46,709: [DEBUG: __main__.<module>] student export file chosen: /Users/aaronciuffo/Downloads/student.export (4).text
2018-09-26 19:44:46,710: [DEBUG: __main__.fileToList] inputfile = /Users/aaronciuffo/Downloads/student.export (4).text
2018-09-26 19:44:46,711: [DEBUG: __main__.fileToList] reading file: /Users/aaronciuffo/Downloads/student.expor


 Continue with the configuration above? [Y/n] 
2018-09-26 19:44:47,646: [DEBUG: humanfriendly.prompts.prompt_for_confirmation] Default choice selected by empty reply ('granted').


In [ ]:
for each in logger.root.handlers:
    if 'FileHandler' in str(each.format.im_class):
        print each.baseFilename
        

In [ ]:
# foo = createFolders(myDrive=myDrive, parentFolder='1Uzp9FFXNEc-_sO8ApVAaU08EY0ekmzwO', teamdriveID='0ADVhiLOX2_ufUk9PVA', studentexport_csv=studentexport_csv, header_map=headerMap )
foo = createFolders(myDrive=myDrive, parentFolder=folderID, teamdriveID=teamdriveID, studentexport_csv=studentexport_csv, header_map=headerMap )



In [ ]:
myDrive.search(name='Misc Junk', fuzzy=True)

In [ ]:
# foo['100000'] = None
# failures = 0
# for each in foo:
#     if not foo[each]:
#         print 'student not processed due to errors: {}'.format(each)
#         failures += 1
# if failures > 0:
#     print 'Please re-run the program with the same data. If this message appears again, please check the logs [[path to logs]]'

# urlFormatter = '<a href={}>Student Portfolio for {}</a>'
# studentData_header = ['LastFirst', 'Student_Number', 'StudentURL']
# studentData_list = []
# studentData_list.append(studentData_header)

# for each in foo:
#     if foo[each]:
#         # LastFirst, Student Number, URL
#         studentData_list.append([foo[each]['LastFirst'], each, 
#                                  urlFormatter.format(foo[each]['webViewLink'], foo[each]['LastFirst'])])
        


# with open('./foo.tsv', mode='w') as tsv_file:
#     csv_writer = csv.writer(tsv_file, delimiter='\t')
#     csv_writer.writerows(studentData_list)